In [92]:
import pandas as pd

df = pd.read_pickle('../data/clean_df.pkl')


In [93]:
drop_col = ['No_disposition', 'Section', 'Nombre_de_lots','No_plan','Nature_culture','Departement','Code_postal','year','day','code_commune_INSEE']

df = df.drop(columns=drop_col)

print(df.columns)
df.head()


Index(['Valeur_fonciere', 'Commune', 'Code_type_local', 'Surface_reelle_bati',
       'Nombre_pieces_principales', 'Surface_terrain', 'month', 'latitude',
       'longitude', 'niveau_vie_commune'],
      dtype='object')


,Valeur_fonciere,Commune,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,month,latitude,longitude,niveau_vie_commune
0,45000.0,SAINT DENIS,2,20.0,1.0,NaN,8,-20.932971,55.446867,19781.428571
1,100000.0,HYERES,2,52.0,3.0,NaN,3,43.101871,6.188985,19910.333333
2,102000.0,ORCHIES,1,54.0,2.0,103.0,3,50.468449,3.238907,19646.666667
3,297000.0,VINCENNES,2,46.0,3.0,NaN,9,48.847373,2.437987,30584.000000
4,74000.0,NANTES,2,36.0,2.0,NaN,6,47.231636,-1.548310,20725.652174


In [94]:
# # Encoder données
# from sklearn.preprocessing import LabelEncoder

# colonnes_catégorielles = list(
#     df_without_nan.select_dtypes(include=['object']).columns)

# print(colonnes_catégorielles)

# label_encoder = LabelEncoder()

# for col in colonnes_catégorielles:
#     df_without_nan[col] = label_encoder.fit_transform(df_without_nan[col])

##df_without_nan = pd.get_dummies(df_without_nan, columns=colonnes_non_numeriques.columns)

#df_without_nan = pd.get_dummies(
#    df_without_nan, columns=['Commune'])


# print(df_without_nan.info())
# df_without_nan.head()


In [95]:
X = df.drop(columns=['Code_type_local'])
y = df['Code_type_local']
print(y)

0          2
1          2
2          1
3          2
4          2
          ..
3778287    2
3778288    1
3778289    1
3778290    4
3778291    1
Name: Code_type_local, Length: 3778292, dtype: int32


In [96]:
# # SÉLECTION DU MEILLEUR NOMBRE DE VARIABLES 

# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import cross_val_score
# import numpy as np

# # # Supposons que X contient vos caractéristiques et y contient votre variable cible
# # # Créez un modèle DecisionTreeClassifier
# modele_decision_tree = DecisionTreeClassifier()

# # # Initialisez une liste pour stocker les scores de cross-validation
# scores = []

# # # Essayez différentes valeurs de k et effectuez une validation croisée pour chaque valeur de k
# # # Essayez toutes les valeurs de k de 1 à nombre total de caractéristiques
# for k in range(1, X.shape[1] + 1):
#     k_best = SelectKBest(score_func=chi2, k=k)
#     X_selected = k_best.fit_transform(X, y)
#     # Utilisez une validation croisée avec 5 plis
#     score = np.mean(cross_val_score(modele_decision_tree, X_selected, y, cv=5))
#     scores.append(score)

# # # Trouvez la valeur de k qui donne le meilleur score
# # # Ajoutez 1 car l'indexation commence à partir de 0
# meilleur_k = np.argmax(scores) + 1
# # # Obtenez le meilleur score correspondant à la meilleure valeur de k
# meilleur_score = scores[meilleur_k - 1]

# print("Meilleur nombre de caractéristiques : {}".format(meilleur_k))
# print("Meilleur score de cross-validation : ",meilleur_score )


In [97]:
# SÉLECTION DU MEILLEUR NOMBRE DE VARIABLES

# from sklearn.feature_selection import SelectKBest, chi2

# # # Créez un objet SelectKBest avec le meilleur nombre de caractéristiques trouvé précédemment
# k_best = SelectKBest(score_func=chi2, k=5)
# X_selected = k_best.fit_transform(X, y)

# # # Obtenez les indices des caractéristiques sélectionnées
# indices_caracteristiques_selectionnees = k_best.get_support()

# # # Obtenez les noms des caractéristiques sélectionnées à partir du DataFrame original
# caracteristiques_selectionnees = X.columns[indices_caracteristiques_selectionnees]

# # # Affichez les noms des caractéristiques sélectionnées
# print("Caractéristiques sélectionnées : ")
# print(caracteristiques_selectionnees)


In [98]:
# Filling Nans Columns


from sklearn.impute import SimpleImputer

# Supposons que votre DataFrame soit nommé 'df'

# Colonnes à imputer
colonnes_a_imputer = ['Surface_reelle_bati','Nombre_pieces_principales','Surface_terrain','niveau_vie_commune']

# Créez un objet SimpleImputer avec la stratégie 'median'
imputer = SimpleImputer(strategy='median')
imputer.fit(df[colonnes_a_imputer])


# Export Imputer

import pickle as pkl

with open('../model/imputer.pkl','wb') as fichier_pickler:
    pkl.dump(imputer,fichier_pickler)

    
# Appliquez l'imputation sur les colonnes sélectionnées
df[colonnes_a_imputer] = imputer.transform(df[colonnes_a_imputer])

In [99]:
# CLASSIFICATION TRAINING 'Code_type_local'


# Import train_test_split function
from sklearn.model_selection import train_test_split
# ########### Random forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

print(df.columns)

# Random forest mieux mais 8min de calcul
caracteristiques_selectionnees = ['Surface_reelle_bati', 'Nombre_pieces_principales','Surface_terrain'] #test

X = df[caracteristiques_selectionnees]
y = df['Code_type_local']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


modele_decision_tree = DecisionTreeClassifier(random_state=42)

modele_decision_tree.fit(X_train, y_train)

# Obtenez l'importance des variables
importances = modele_decision_tree.feature_importances_
print(importances)

y_pred = modele_decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Index(['Valeur_fonciere', 'Commune', 'Code_type_local', 'Surface_reelle_bati',
       'Nombre_pieces_principales', 'Surface_terrain', 'month', 'latitude',
       'longitude', 'niveau_vie_commune'],
      dtype='object')
[0.58565814 0.12766881 0.28667305]
Accuracy: 0.9513157616137092


In [100]:
X

,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain
0,20.0,1.0,488.0
1,52.0,3.0,488.0
2,54.0,2.0,103.0
3,46.0,3.0,488.0
4,36.0,2.0,488.0
...,...,...,...
3778287,67.0,3.0,488.0
3778288,50.0,1.0,300.0
3778289,82.0,6.0,95.0
3778290,100.0,0.0,175.0


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3778292 entries, 0 to 3778291
Data columns (total 10 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Valeur_fonciere            float64
 1   Commune                    object 
 2   Code_type_local            int32  
 3   Surface_reelle_bati        float64
 4   Nombre_pieces_principales  float64
 5   Surface_terrain            float64
 6   month                      object 
 7   latitude                   float64
 8   longitude                  float64
 9   niveau_vie_commune         float64
dtypes: float64(7), int32(1), object(2)
memory usage: 273.8+ MB


In [102]:
df.head()

,Valeur_fonciere,Commune,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,month,latitude,longitude,niveau_vie_commune
0,45000.0,SAINT DENIS,2,20.0,1.0,488.0,8,-20.932971,55.446867,19781.428571
1,100000.0,HYERES,2,52.0,3.0,488.0,3,43.101871,6.188985,19910.333333
2,102000.0,ORCHIES,1,54.0,2.0,103.0,3,50.468449,3.238907,19646.666667
3,297000.0,VINCENNES,2,46.0,3.0,488.0,9,48.847373,2.437987,30584.000000
4,74000.0,NANTES,2,36.0,2.0,488.0,6,47.231636,-1.548310,20725.652174


In [103]:
pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)
print(len(df))

df.head()

Valeur_fonciere              0.0
Commune                      0.0
Code_type_local              0.0
Surface_reelle_bati          0.0
Nombre_pieces_principales    0.0
Surface_terrain              0.0
month                        0.0
latitude                     0.0
longitude                    0.0
niveau_vie_commune           0.0
dtype: float64
3778292


,Valeur_fonciere,Commune,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,month,latitude,longitude,niveau_vie_commune
0,45000.0,SAINT DENIS,2,20.0,1.0,488.0,8,-20.932971,55.446867,19781.428571
1,100000.0,HYERES,2,52.0,3.0,488.0,3,43.101871,6.188985,19910.333333
2,102000.0,ORCHIES,1,54.0,2.0,103.0,3,50.468449,3.238907,19646.666667
3,297000.0,VINCENNES,2,46.0,3.0,488.0,9,48.847373,2.437987,30584.000000
4,74000.0,NANTES,2,36.0,2.0,488.0,6,47.231636,-1.548310,20725.652174


In [104]:
# Adding Variable 'Prix_moyen_m2'

import numpy as np

# Groupez les données par 'Commune' et calculez la somme de 'Surface_reel' et 'Valeur_fonciere' pour chaque commune
groupe_commune = df.groupby('Commune').agg(
    {'Surface_reelle_bati': 'sum', 'Valeur_fonciere': 'sum'})

# Calculez le prix moyen au mètre carré par commune
groupe_commune['Prix_moyen_m2'] = groupe_commune['Valeur_fonciere'] / groupe_commune['Surface_reelle_bati']

# Exporter les données de Prix_moyen_m2
print(groupe_commune)
groupe_commune.to_pickle('../data/prix_moyen_m2.pkl')



# Fusionnez le DataFrame original avec le DataFrame groupe_commune sur la colonne 'Commune'
df = pd.merge(df, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')



# Remplace les inf par NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Supprime les lignes avec NaN dans 'Prix_moyen_m2'
df = df.dropna(subset=['Prix_moyen_m2'], how='any')

#print(df.info())



                      Surface_reelle_bati  Valeur_fonciere  Prix_moyen_m2
Commune                                                                  
AASAINT                             289.0        418600.00    1448.442907
ABAINVILLE                          949.0        810423.52     853.976312
ABANCOURT                          5845.0       7734154.00    1323.208554
ABAUCOURT                          1807.0       2508440.00    1388.179303
ABAUCOURT HAUTECOURT                496.0        294100.00     592.943548
...                                   ...              ...            ...
ZUANI                               112.0        206400.00    1842.857143
ZUDAUSQUES                         5770.0       8944037.00    1550.093068
ZUTKERQUE                          9162.0      14510044.00    1583.720148
ZUYDCOOTE                          6997.0      17719934.00    2532.504502
ZUYTPEENE                          3720.0       6246910.00    1679.276882

[30248 rows x 3 columns]


In [105]:
# Export final data

df.to_pickle('../data/final_data.pkl')


In [106]:
# Export classif model 

with open('../model/modele_classification.pkl','wb') as fichier_pickler:
    pkl.dump(modele_decision_tree,fichier_pickler)
